<h1>Credit Card Lead Prediction</h1>

<h2>Problem Statement</h2>

Happy Customer Bank is a mid-sized private bank that deals in all kinds of banking products, like Savings accounts, Current accounts, investment products, credit products, among other offerings.

The bank also cross-sells products to its existing customers and to do so they use different kinds of communication like tele-calling, e-mails, recommendations on net banking, mobile banking, etc. 

In this case, the Happy Customer Bank wants to cross sell its credit cards to its existing customers. The bank has identified a set of customers that are eligible for taking these credit cards.

Now, the bank is looking for your help in identifying customers that could show higher intent towards a recommended credit card, given:

* Customer details (gender, age, region etc.)
* Details of his/her relationship with the bank (Channel_Code, Vintage, Avg_Asset_Value etc.)

<h1>Import Libraries</h1>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer
from scipy.stats import chi2_contingency, chi2


# Model Building
from sklearn.naive_bayes import GaussianNB                                 # Naive Bayes
from sklearn.linear_model import LogisticRegression                        # Logistic Regression
from sklearn.neighbors import KNeighborsClassifier                         # KNN
from sklearn.tree import DecisionTreeClassifier                            # Decision Tree
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier    # Random Forest, AdaBoost
from sklearn.svm import SVC                                                # Support Vector Machines (SVM)

from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_auc_score

# -------------------------------------------------------------------------------------------------

import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all' 

import winsound
duration=2000    # milliseconds
freq=440         # Hz
# winsound.Beep(freq,duration)         # Use this command wherever required for alarm

# Exploratory Data Analysis

In [ ]:
# Load the data set
data = pd.read_csv('train_s3TEQDk.csv')

# Check the dimension of data set
data.shape
data.head()

In [ ]:
# Check datatype of each variable
data.info()

In [ ]:
# Summary of continuous variables
data.describe()

In [ ]:
# Summary of categorical variables
data.describe(include='object')

The ID column has unique ID for all the customers, so this does not give any information for prediction of lead.
Hence we delete this column.

In [ ]:
# Delete 'ID' column 
data.drop('ID',axis=1,inplace=True)
data.head()

In [ ]:
# Check for missing values in data set
data.isnull().sum()/data.shape[0] *100

Here, Credit_Product is the only variable with 11.9% missing values. Deleting these missing values would result in a lot of information being lost. So, these values need to be imputed. Being categorical variable, the obvious choice is imputation with modal value. But imputation with modal value changes the distribution of the data, so other method of imputation must be implemented. Here, the values will be imputed with KNN imputaion, for which all the variables must be numeric as KNN imputer uses distance function for imputation. Before imputation let's explore the dataset.  

# Data Visualization

## Exploring Continuous Variables

In [ ]:
# Continuous features
conti_features = data.select_dtypes(exclude=object).columns
conti_features
data[conti_features].head()

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(18,5))
fig.suptitle("Interaction between Continuous Features & Lead", fontsize=30, fontweight=15)

sns.boxplot(x="Is_Lead", y="Age", data=data, ax=axes[0]);
sns.boxplot(x="Is_Lead", y="Vintage", data=data, ax=axes[1]);
sns.boxplot(x="Is_Lead", y="Avg_Account_Balance", data=data, ax=axes[2]);

The above plot shows that all the variables are positively skewed with little or no ouliers as such except for the variable 'Avg_Account_Balance' which has many outliers.

In [ ]:
# Distribution of continuous variables
data[conti_features[:-1]].plot(kind='density', subplots=True, layout=(1,3), figsize=(15,5), sharex=False, sharey=False, 
                           title="Distribution of Age, Vintage & Avg_Account_Balance")
plt.show();

In [ ]:
# Scatter plot of continuous variables
from pandas.plotting import scatter_matrix
scatter_matrix(data[conti_features[:-1]])
plt.show();

There seems to be some positive correlation between Age & Vintage. Let's verify.

In [ ]:
# Correlations between continuous variables
print("Correlation(Age,Vintage) :",data['Age'].corr(data['Vintage']))
print("Correlation(Vintage,Avg_Account_Balance) :",data['Vintage'].corr(data['Avg_Account_Balance']))
print("Correlation(Age,Avg_Account_Balance) :",data['Age'].corr(data['Avg_Account_Balance']))

Here, it is verified that 'Age' & 'Vintage' are positively correlated with corr=0.63, but the correlation is not too high that one of them can be deleted to avoid the problem of multicollinearity.

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(18,5))
fig.suptitle("Violin plot for better visualiusation of Distribution of Continuous Features", fontsize=20, fontweight=15)

sns.violinplot(x=data.Age, ax=axes[0]);
sns.violinplot(x=data.Vintage, ax=axes[1]);
sns.violinplot(x=data.Avg_Account_Balance, ax=axes[2]);

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(18,5))
fig.suptitle("Violin plot after log transformation of Continuous Features", fontsize=20, fontweight=15)

dis_age = np.log(data.Age)
dis_vin = np.log(data.Vintage)
dis_aab = np.log(data.Avg_Account_Balance)


sns.violinplot(x=dis_age, ax=axes[0]);
sns.violinplot(x=dis_vin, ax=axes[1]);
sns.violinplot(x=dis_aab, ax=axes[2]);

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(18,5))
fig.suptitle("Box-plot of Continuous Features", fontsize=30, fontweight=15)

sns.boxplot(x=dis_age, ax=axes[0]);
sns.boxplot(x=dis_vin, ax=axes[1]);
sns.boxplot(x=dis_aab, ax=axes[2]);

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(18,5))
fig.suptitle("Interaction between Continuous Features & Lead", fontsize=30, fontweight=15)

sns.boxplot(x="Age", data=data, ax=axes[0]);
sns.boxplot(x="Vintage", data=data, ax=axes[1]);
sns.boxplot(x="Avg_Account_Balance", data=data, ax=axes[2]);

## Exploring Categorical Variables

In [ ]:
# Extract all categorical features
categ_features = data.select_dtypes(include=object).columns
categ_features

In [ ]:
# Number of Males & Females
data.Gender.value_counts()

In [ ]:
# Number of Active & Inactive customers
data.Is_Active.value_counts()

In [ ]:
# Number of customers with & without Credit Product
data.Credit_Product.value_counts()

In [ ]:
# Number of unique Region_Code
print(data.Region_Code.unique())
print(data.Region_Code.value_counts().shape)

In [ ]:
# Occupation of customers & their number
data.Occupation.value_counts()

In [ ]:
# Unique Channel code & their counts
data.Channel_Code.value_counts()

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(18,10))
fig.suptitle("Interaction between Categorical Features & Lead", fontsize=30, fontweight=15)

sns.countplot(data.Gender,hue=data.Is_Lead, ax=axes[0,0], palette='ocean');
sns.countplot(data.Is_Active,hue=data.Is_Lead, ax=axes[0,1], palette='spring');
sns.countplot(data.Credit_Product,hue=data.Is_Lead, ax=axes[0,2], palette='summer');
sns.countplot(data.Region_Code,hue=data.Is_Lead, ax=axes[1,0], palette='autumn');
sns.countplot(data.Occupation,hue=data.Is_Lead, ax=axes[1,1], palette='twilight');
sns.countplot(data.Channel_Code,hue=data.Is_Lead, ax=axes[1,2], palette='rocket');

# Encoding Categorical Variable

## Testing significance of 'Region Code'

**Significance of Region_Code**

The variable 'Region_Code' has 35 unique categories, so encoding them as dummy variables would create 34 new variables which increases the dimension and might also hamper the model performance due to sparse data. Frequency encoding is a better solution to go ahead with. Before encoding it would be wise to see if this variable is significant in prediction of 'Is_Lead'.

In [ ]:
# Chi-Square test for significance of Region_Code in predicting Lead
observed_values = pd.crosstab(data.Region_Code,data.Is_Lead)
statistic, pvalue, dof, expected = chi2_contingency(observed_values)

In [ ]:
print("p-value : {} \nChi-Square statistic : {} \nDegree of Freedom : {}".format(pvalue,statistic,dof))

In [ ]:
# Interpretation of test-statistic
prob=0.95  # (1-alpha)% 
critical = chi2.ppf(prob, dof)
print("Chi-Square statistic : {} and critical value : {}.".format(statistic,critical))
if abs(statistic) >= critical:
    print('Dependent (reject H0)')
else:
    print('Independent (fail to reject H0)')

- **If Statistic >= Critical Value:** significant result, reject null hypothesis (H0), dependent.

- **If Statistic < Critical Value:** not significant result, fail to reject null hypothesis (H0), independent.

Here, test statistic is greater then critical value. Therefore, the null-hypothesis is rejected.

In [ ]:
# Interpretation of p-value
alpha = 0.05
print("P-value : {} and alpha : {}.".format(pvalue,alpha))
if pvalue <= alpha:
    print('Dependent (reject H0)')
else:
    print('Independent (fail to reject H0)')

- **If p-value <= alpha:** significant result, reject null hypothesis (H0), dependent.

- **If p-value > alpha:** not significant result, fail to reject null hypothesis (H0), independent.

Here, as p-value is less than alpha=0.05. Therefore, the null-hypothesis H0 of Independence is rejected at 5% level of significance. 

**Region_Code is a significant feature in predicting Leads**.

## Encoding Categorical Variables 

In [ ]:
# Create copies of Data

data1 = data.copy()
data2 = data.copy()
data3 = data.copy()

data1.head()

In [ ]:
# Function for encodings

def encode(data):
    dict_Region = data.Region_Code.value_counts().to_dict()
    data['Region_Code'] = data.Region_Code.map(dict_Region)
    
    data['Credit_Product'] = LabelEncoder().fit_transform(data.Credit_Product)
    #data['Credit_Product'] = data.Credit_Product.map({'Yes': 1, 'No': 0},na_action=None)
    data['Gender'] = data.Gender.map({'Male': 1, 'Female': 0})
    data['Is_Active'] = data.Is_Active.map({'Yes' : 1, 'No' : 0})
    data['Channel_Code'] = LabelEncoder().fit_transform(data.Channel_Code)
    
    dict_Occupation = data.Occupation.value_counts().to_dict()
    data['Occupation'] = data.Occupation.map(dict_Occupation)

In [ ]:
encode(data1)
data1.head()

In [ ]:
corr = data1.corr()
sns.heatmap(corr);

In [ ]:
# Fill missing values with 'mode'
#data['Credit_Product'].fillna(data['Credit_Product'].mode()[0], inplace=True)

# Train Test Split

In [ ]:
# Split data into training & testing
train, test = train_test_split(data1, test_size=0.2, random_state=19, stratify=data1['Is_Lead'])
train.shape; test.shape

# Train Valdation split

In [ ]:
train.head()

In [ ]:
# Separate Features & Target
y = train['Is_Lead']
X = train.drop('Is_Lead',axis=1)
X.shape; y.shape

In [ ]:
# Split data into training & validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=19)
X_train.shape; X_val.shape; y_train.shape; y_val.shape

# Model Building

In [ ]:
# Model instances

lr = LogisticRegression()
nb = GaussianNB()
knn = KNeighborsClassifier(n_neighbors=5)
dtree = DecisionTreeClassifier(max_depth=5, random_state=101, max_features=None, min_samples_leaf=15)
rfm = RandomForestClassifier(n_estimators=70, oob_score=True, n_jobs=-1,
                             random_state=101, max_features=None, min_samples_leaf=15)
ab = AdaBoostClassifier(n_estimators=50,learning_rate=1, random_state=14)          # base_estimator --> Decision tree (default)
svc = SVC(kernel='linear', probability=True,random_state=101)

### Decision Tree

In [ ]:
%%time

# Fit model
dtree.fit(X_train,y_train)

# Predict
y_hat = dtree.predict(X_val)

# Classification
cr = classification_report(y_true=y_val,y_pred=y_hat)
print(cr)

In [ ]:
# Evaluation
acc = accuracy_score(y_true=y_val,y_pred=y_hat)
pred_prob = dtree.predict_proba(X_val)
auc_score = roc_auc_score(y_val, pred_prob[:,1])

TN, FP, FN, TP = confusion_matrix(y_true=y_val,y_pred=y_hat).ravel()
sens = TP / (TP+FN)
spec = TN / (TN+FP)
prec = TP / (TP+FP)
f1 = 2*prec*sens / (prec + sens)

# Result
result = pd.DataFrame({'Model': 'Decision Tree',
                           'Accuracy Score':acc,
                           'AUC Score': auc_score,
                           'F1 Score': f1,
                           'Precision': prec,
                           'Sensitivity(Recall)': sens,
                           'Specificity': spec }, index=[0])
result

### SVM Classifier

In [ ]:
svc = SVC(kernel='rbf')
svc.fit(X_train,y_train)

In [ ]:
winsound.Beep(freq,duration)

In [ ]:
y_hat = svc.predict(X_val)

In [ ]:
acc = accuracy_score(y_true=y_val,y_pred=y_hat)
pred_prob = svc.predict_proba(X_val)
auc_score = roc_auc_score(y_val, pred_prob[:,1])

TN, FP, FN, TP = confusion_matrix(y_true=y_val,y_pred=y_hat).ravel()
sens = TP / (TP+FN)
spec = TN / (TN+FP)
prec = TP / (TP+FP)
f1 = 2*prec*sens / (prec + sens)

In [ ]:
result = pd.DataFrame({'Model': 'Support Vector Classifier',
                           'Accuracy Score':acc,
                           'AUC Score': auc_score,
                           'F1 Score': f1,
                           'Precision': prec,
                           'Sensitivity(Recall)': sens,
                           'Specificity': spec }, index=[0])
result

### Naive Bayes

In [ ]:
# Fit model
nb.fit(X_train,y_train)

# Predict
y_hat = nb.predict(X_val)

# Classification Report
cr = classification_report(y_true=y_val,y_pred=y_hat)
print(cr)

In [ ]:
# Evaluation
acc = accuracy_score(y_true=y_val,y_pred=y_hat)
pred_prob = nb.predict_proba(X_val)
auc_score = roc_auc_score(y_val, pred_prob[:,1])

TN, FP, FN, TP = confusion_matrix(y_true=y_val,y_pred=y_hat).ravel()
sens = TP / (TP+FN)
spec = TN / (TN+FP)
prec = TP / (TP+FP)
f1 = 2*prec*sens / (prec + sens)

# Result
result = pd.DataFrame({'Model': 'Naive Bayes',
                           'Accuracy Score':acc,
                           'AUC Score': auc_score,
                           'F1 Score': f1,
                           'Precision': prec,
                           'Sensitivity(Recall)': sens,
                           'Specificity': spec }, index=[0])
result

# Multiple Models

In [ ]:
algos = [lr, nb, sgd, knn, dtree, rfm, ab]
names =['Logistic Regression','Naive Bayes','SGD','KNN','Decision Tree','Random Forest', 'AdaBoost']

In [ ]:
auc_list=[]
acc_list=[]
sensitivity=[]
specificity=[]
precision=[]
f1_score=[]

for name in algos:
    
    model = name
    model.fit(X_train,y_train)
    y_hat = model.predict(X_val)

    pred_prob = model.predict_proba(X_val)

    auc = roc_auc_score(y_true=y_val, y_score=pred_prob[:,1])
    acc = accuracy_score(y_true=y_val,y_pred=y_hat)
    TN, FP, FN, TP = confusion_matrix(y_true=y_val,y_pred=y_hat).ravel()

    sens = TP / (TP+FN)
    spec = TN / (TN+FP)       
    prec = TP / (TP+FP)
    f1 = 2*prec*sens / (prec + sens)

    auc_list.append(auc)          # AUC Score 
    acc_list.append(acc)          # Accuracy Score
    sensitivity.append(sens)      # Sensitivity/Recall 
    specificity.append(spec)      # Specificity 
    precision.append(prec)        # Precision
    f1_score.append(f1)           # F1 Score
    winsound.Beep(freq,duration)

In [ ]:
evaluation = pd.DataFrame({'Model': names,
                           'Accuracy Score':acc_list,
                           'AUC Score': auc_list,
                           'F1 Score': f1_score,
                           'Precision': precision,
                           'Sensitivity(Recall)': sensitivity,
                           'Specificity': specificity })
evaluation

In [ ]:
evaluation

# Visualization

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_reduced = pca.fit_transform(X_train)
X_reduced[:10]

In [ ]:
np.round(pca.explained_variance_ratio_,2)

In [ ]:
np.round(1 - pca.explained_variance_ratio_.sum(),4)

In [ ]:
def plot_after_dim_reduce(X_reduced):
    for i in range(len(y)):
        if y[i] == 0:
            plt.scatter(X_reduced[i, 0], X_reduced[i, 1], c='b')
        elif y[i] == 1:
            plt.scatter(X_reduced[i, 0], X_reduced[i, 1], c='r')
    plt.grid(True)
    plt.show()

In [ ]:
plot_after_dim_reduce(X_reduced)

In [ ]:
# IncrementalPCA

from sklearn.decomposition import IncrementalPCA

n_batches = 50
inc_pca = IncrementalPCA(n_components=2)
for X_batch in np.array_split(X, n_batches):
    inc_pca.partial_fit(X_batch)

X_reduced_inc_pca = inc_pca.fit_transform(X)

In [ ]:
plot_after_dim_reduce(X_reduced_inc_pca)

In [ ]:
# KernelPCA
from sklearn.decomposition import KernelPCA

rbf_pca = KernelPCA(n_components=2, kernel='rbf', gamma=0.03)
X_reduced_kernel_rbf = rbf_pca.fit_transform(X)

In [ ]:
plot_after_dim_reduce(X_reduced_kernel_rbf)

In [ ]:
#
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

clf = Pipeline([
        ("kpca", KernelPCA(n_components=2)),
        ("log_reg", LogisticRegression())
    ])

param_grid = [{
        "kpca__gamma": np.linspace(0.03, 0.05, 10),
        "kpca__kernel": ["rbf", "sigmoid", "poly", "linear"]
    }]

grid_search = GridSearchCV(clf, param_grid, cv=10);
grid_search.fit(X, y);

In [ ]:
print(grid_search.best_params_)

In [ ]:
lin_pca = KernelPCA(n_components=2, kernel='linear', gamma=0.03)
X_reduced_kernel_lin = lin_pca.fit_transform(X)

In [ ]:
plot_after_dim_reduce(X_reduced_kernel_lin)

In [ ]:
# LocallyLinearEmbedding
from sklearn.manifold import LocallyLinearEmbedding

lle = LocallyLinearEmbedding(n_components=2, random_state=42)
X_reduced_lle = lle.fit_transform(X)

In [ ]:
plot_after_dim_reduce(X_reduced_lle)

In [ ]:
# MDS
from sklearn.manifold import MDS

mds = MDS(n_components=2, random_state=42)
X_reduced_mds = mds.fit_transform(X)

In [ ]:
plot_after_dim_reduce(X_reduced_mds)

In [ ]:
# Isomap
from sklearn.manifold import Isomap

isomap = Isomap(n_components=2)
X_reduced_isomap = isomap.fit_transform(X)

In [ ]:
plot_after_dim_reduce(X_reduced_isomap)

In [ ]:
# 
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, random_state=42)
X_reduced_tsne = tsne.fit_transform(X)

In [ ]:
plot_after_dim_reduce(X_reduced_tsne)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
# LogisticRegression
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(random_state=42)
log_reg.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score, auc, confusion_matrix, roc_curve

y_pred = log_reg.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print('Accuracy : {}'.format(acc))

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
from sklearn.metrics import roc_auc_score
import scikitplot as skplt

y_pred_proba = log_reg.predict_proba(X_test)
skplt.metrics.plot_precision_recall(y_test, y_pred_proba)
plt.show()

In [ ]:
# Logistic Regression with dimensinality reduction (Linear PCA)

X_train_dr, X_test_dr, y_train_dr, y_test_dr = train_test_split(X_reduced_kernel_lin, y, test_size=0.2)
log_reg = LogisticRegression(random_state=42)
log_reg.fit(X_train_dr, y_train_dr)

In [ ]:
y_pred_dr = log_reg.predict(X_test_dr)
acc_dr = accuracy_score(y_test_dr, y_pred_dr)
print('Accuracy : {}'.format(acc_dr))

In [ ]:
confusion_matrix(y_test_dr, y_pred_dr)

In [ ]:
# Logistic Regression with dimensinality reduction (MDS)

X_train_mds, X_test_mds, y_train_mds, y_test_mds = train_test_split(X_reduced_mds, y, test_size=0.2)
log_reg = LogisticRegression(random_state=42)
log_reg.fit(X_train_mds, y_train_mds)

In [ ]:
y_pred_mds = log_reg.predict(X_test_mds)
acc_mds = accuracy_score(y_test_mds, y_pred_mds)
print('Accuracy : {}'.format(acc_mds))

In [ ]:
confusion_matrix(y_test_mds, y_pred_mds)

In [ ]:
# Logistic Regression with dimensinality reduction (t-SNE)

X_train_tsne, X_test_tsne, y_train_tsne, y_test_tsne = train_test_split(X_reduced_tsne, y, test_size=0.2)
log_reg = LogisticRegression(random_state=42)
log_reg.fit(X_train_tsne, y_train_tsne)

In [ ]:
y_pred_tsne = log_reg.predict(X_test_tsne)
acc_tsne = accuracy_score(y_test_tsne, y_pred_tsne)
print('Accuracy : {}'.format(acc_tsne))

In [ ]:
confusion_matrix(y_test_tsne, y_pred_tsne)

# Cross Validation

In [ ]:
# Cross-Validation

from sklearn.cross_validation import KFold, RepeatedKFold, cross_val_score

# Prepare the cross-validation procedure
cv = KFold(n_splits=10, random_state=1, shuffle=True)
repcv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

# Create Model
model = LogisticRegression()

# Evaluate Model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)

print(cross_val_score(model, X_train, y_train, cv=4))
print(cross_val_score(model, X, y, cv=2))

# Standardization & Normalization

In [ ]:
# Standardization

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_data)
standardized_X = scaler.transform(X_data)
standardized_X_test = scaler.transform(X_test)

# Normalization

from sklearn.preprocessing import Normalizer
scaler = Normalizer().fit(X_data)
normalized_X = scaler.transform(X_data)
normalized_X_test = scaler.transform(X_test)

# Imputing missing values 

In [ ]:
# Train KNN imputer on train set
knn = KNNImputer(n_neighbors=10)
knn.fit(train)

impute_train= knn.transform(train)


impute_train.shape; impute_train.dtypes

# Impute test set by KNN imputer trained on train set
impute_test= knn.transform(test)
winsound.Beep(freq,duration)

df_train = pd.DataFrame(impute_train, columns= data.columns)
df_train.head()

df_test = pd.DataFrame(impute_test, columns= data.columns)
df_test.head()

In [ ]:
# matplotlib
import matplotlib.pyplot as plt
plt.style.use('seaborn')

# plot roc curves
plt.plot(fpr1, tpr1, linestyle='--',color='orange', label='Logistic Regression')
plt.plot(fpr2, tpr2, linestyle='--',color='green', label='KNN')
plt.plot(p_fpr, p_tpr, linestyle='--', color='blue')
# title
plt.title('ROC curve')
# x label
plt.xlabel('False Positive Rate')
# y label
plt.ylabel('True Positive rate')

plt.legend(loc='best')
plt.savefig('ROC',dpi=300)
plt.show();